In [1]:
import sys
import os
sys.path.append(os.path.abspath("Pytorch/"))
sys.path.append(os.path.abspath("models/"))

In [2]:
from CorrelatedFBSNNs import *
from CorrelatedBlackScholesBarenblatt2D import *

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 100  # number of trajectories (batch size)
N = 50  # number of time snapshots
D = 2 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1.0, 0.5] * int(D / 2))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
model = BlackScholesBarenblatt(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 2 * 10**4
lr = 1e-3

In [8]:
n_iter = 5 * 10**3
lr = 1e-5

In [9]:
tot = time.time()
print(model.device)
graph = model.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 19900, Loss: 3.218e-01, Y0: 1.567, Time: 0.18, Learning Rate: 1.000e-05
It: 20000, Loss: 2.439e-01, Y0: 1.566, Time: 17.59, Learning Rate: 1.000e-05
It: 20100, Loss: 2.083e-01, Y0: 1.570, Time: 17.74, Learning Rate: 1.000e-05
It: 20200, Loss: 2.509e-01, Y0: 1.560, Time: 17.53, Learning Rate: 1.000e-05
It: 20300, Loss: 2.430e-01, Y0: 1.566, Time: 17.70, Learning Rate: 1.000e-05
It: 20400, Loss: 1.641e-01, Y0: 1.563, Time: 17.87, Learning Rate: 1.000e-05
It: 20500, Loss: 2.173e-01, Y0: 1.567, Time: 22.81, Learning Rate: 1.000e-05
It: 20600, Loss: 1.465e-01, Y0: 1.559, Time: 23.00, Learning Rate: 1.000e-05
It: 20700, Loss: 2.125e-01, Y0: 1.567, Time: 18.09, Learning Rate: 1.000e-05
It: 20800, Loss: 1.979e-01, Y0: 1.564, Time: 17.65, Learning Rate: 1.000e-05
It: 20900, Loss: 3.953e-01, Y0: 1.568, Time: 20.22, Learning Rate: 1.000e-05
It: 21000, Loss: 1.440e-01, Y0: 1.564, Time: 20.17, Learning Rate: 1.000e-05
It: 21100, Loss: 1.625e-01, Y0: 1.562, Time: 23.05, Learning Rate: 1.0

In [4]:
model.load_model("models/BlackScholesBarenblatt2DCorr.pth")

In [5]:
graph = model.iteration, model.training_loss

In [10]:
np.random.seed(42)
t_test, W_test = model.fetch_minibatch()
X_pred, Y_pred = model.predict(Xi, t_test, W_test)
samples = 10

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

Y_test = np.reshape(u_exact(T,np.reshape(t_test[0:M, :, :], [-1, 1]), np.reshape(X_pred[0:M, :, :], [-1, D])),
                    [M, -1, 1])

In [11]:
plt.figure()
plt.plot(graph[0], graph[1])
plt.xlabel('Iterations')
plt.ylabel('Value')
plt.yscale("log")
plt.title('Evolution of the training loss')

plt.figure()
plt.plot(t_test[0:1, :, 0].T, Y_pred[0:1, :, 0].T, 'b', label='Learned $u(t,X_t)$')
plt.plot(t_test[0:1, :, 0].T, Y_test[0:1, :, 0].T, 'r--', label='Exact $u(t,X_t)$')
plt.plot(t_test[0:1, -1, 0], Y_test[0:1, -1, 0], 'ko', label='$Y_T = u(T,X_T)$')

plt.plot(t_test[1:samples, :, 0].T, Y_pred[1:samples, :, 0].T, 'b')
plt.plot(t_test[1:samples, :, 0].T, Y_test[1:samples, :, 0].T, 'r--')
plt.plot(t_test[1:samples, -1, 0], Y_test[1:samples, -1, 0], 'ko')

plt.plot([0], Y_test[0, 0, 0], 'ks', label='$Y_0 = u(0,X_0)$')

plt.xlabel('$t$')
plt.ylabel('$Y_t = u(t,X_t)$')
plt.title(str(D) + '-dimensional Correlated Black-Scholes-Barenblatt, ' + model.mode + "-" + model.activation)
plt.legend()

plt.show()

In [14]:
errors = np.sqrt((Y_test - Y_pred) ** 2 / Y_test ** 2)
mean_errors = np.mean(errors, 0)
std_errors = np.std(errors, 0)

plt.figure()
plt.plot(t_test[0, :, 0], mean_errors, 'b', label='mean')
plt.plot(t_test[0, :, 0], mean_errors + 2 * std_errors, 'r--', label='mean + two standard deviations')
plt.xlabel('$t$')
plt.ylabel('relative error')
plt.title(str(D) + '-dimensional Black-Scholes-Barenblatt, ' + model.mode + "-" + model.activation)
plt.legend()
plt.show()

In [15]:
model.save_model("models/BlackScholesBarenblatt2DCorr.pth")